In [77]:
# dependencies

# basic
from datetime import datetime
import json
import pandas as pd
from pandas import ExcelFile
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import numpy as np
import requests
import seaborn as sns
import time

# nba api
import nba_api
from nba_api.stats.endpoints import BoxScoreAdvancedV2, BoxScoreDefensive, BoxScoreMiscV2, BoxScorePlayerTrackV2, BoxScoreTraditionalV2, BoxScoreUsageV2
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder, leaguegamelog, playergamelog

# sklearn
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, Normalizer, StandardScaler
from sklearn.svm import SVC

# tensorflow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

In [8]:
nbateams = teams.get_teams()

In [30]:
df_current_season_games_full = leaguegamelog.LeagueGameLog().get_data_frames()[0]

In [31]:
df_current_season_games_full

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22019,1610612746,LAC,LA Clippers,0021900002,2019-10-22,LAC vs. LAL,W,240,42,...,34,45,24,8,5,14,25,112,10,1
1,22019,1610612747,LAL,Los Angeles Lakers,0021900002,2019-10-22,LAL @ LAC,L,240,37,...,32,41,20,4,7,15,24,102,-10,1
2,22019,1610612761,TOR,Toronto Raptors,0021900001,2019-10-22,TOR vs. NOP,W,265,42,...,41,57,23,7,3,17,24,130,8,1
3,22019,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,265,43,...,37,53,30,4,9,19,34,122,-8,1
4,22019,1610612755,PHI,Philadelphia 76ers,0021900008,2019-10-23,PHI vs. BOS,W,240,37,...,50,62,24,6,7,17,34,107,14,1
5,22019,1610612738,BOS,Boston Celtics,0021900008,2019-10-23,BOS @ PHI,L,240,33,...,31,41,18,4,2,11,29,93,-14,1
6,22019,1610612752,NYK,New York Knicks,0021900010,2019-10-23,NYK @ SAS,L,240,44,...,34,39,24,16,1,14,32,111,-9,1
7,22019,1610612759,SAS,San Antonio Spurs,0021900010,2019-10-23,SAS vs. NYK,W,240,42,...,41,51,22,6,5,21,18,120,9,1
8,22019,1610612758,SAC,Sacramento Kings,0021900012,2019-10-23,SAC @ PHX,L,240,34,...,35,47,17,5,5,27,24,95,-29,1
9,22019,1610612756,PHX,Phoenix Suns,0021900012,2019-10-23,PHX vs. SAC,W,240,49,...,35,44,31,13,9,14,25,124,29,1


In [29]:
def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result

In [32]:
df_current_season_games = combine_team_games(df_current_season_games_full)

In [36]:
BoxScoreAdvancedV2(game_id = '0021900011').get_data_frames()[0]
# BoxScoreDefensive
# BoxScoreMiscV2
# BoxScorePlayerTrackV2
# BoxScoreUsageV2

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,E_OFF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0021900011,1610612760,OKC,Oklahoma City,1628390,Terrance Ferguson,F,,23:51,81.0,...,33.3,0.000,1.136,0.036,0.037,96.76,96.60,80.50,48,0.023
1,0021900011,1610612760,OKC,Oklahoma City,201568,Danilo Gallinari,F,,28:20,115.6,...,5.6,0.615,0.691,0.250,0.251,97.60,98.28,81.90,59,0.224
2,0021900011,1610612760,OKC,Oklahoma City,203500,Steven Adams,C,,27:42,96.1,...,13.3,0.125,0.141,0.206,0.209,97.49,97.04,80.87,56,-0.037
3,0021900011,1610612760,OKC,Oklahoma City,1628983,Shai Gilgeous-Alexander,G,,36:46,96.8,...,3.6,0.500,0.507,0.325,0.332,99.61,101.18,84.32,77,0.094
4,0021900011,1610612760,OKC,Oklahoma City,101108,Chris Paul,G,,30:07,93.4,...,4.8,0.600,0.656,0.243,0.249,101.27,103.60,86.33,65,0.266
5,0021900011,1610612760,OKC,Oklahoma City,203471,Dennis Schroder,,,30:40,87.0,...,14.3,0.188,0.188,0.133,0.135,106.12,105.65,88.04,67,0.020
6,0021900011,1610612760,OKC,Oklahoma City,203457,Nerlens Noel,,,4:42,36.4,...,0.0,1.000,1.000,0.091,0.091,107.27,102.16,85.14,10,0.409
7,0021900011,1610612760,OKC,Oklahoma City,1629647,Darius Bazley,,,19:15,57.8,...,25.0,0.000,0.000,0.082,0.083,106.57,104.73,87.27,42,-0.098
8,0021900011,1610612760,OKC,Oklahoma City,1628977,Hamidou Diallo,,,23:01,86.1,...,25.0,0.625,0.615,0.203,0.208,110.12,112.58,93.82,53,0.080
9,0021900011,1610612760,OKC,Oklahoma City,203488,Mike Muscala,,,15:36,96.3,...,11.1,0.000,0.347,0.175,0.178,109.10,110.76,92.30,36,0.080


In [46]:
test_array = df_current_season_games['GAME_ID'].unique()[:10]
unique_array = df_current_season_games['GAME_ID'].unique()
test_df = df_current_season_games_full.copy()

In [78]:
df_adv_list = []
df_def_list = []
df_misc_list = []
df_tracking_list = []
df_usage_list = []
df_traditional_list = []

startTime = datetime.now()

game_ok_count = 0
game_no_count = 0

game_problem_list = []

for game in test_array:
  
    try:

        print(f'Searching for game: {game}...')

        df_advanced = BoxScoreAdvancedV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 1.Advanced df found and saved')
        
        df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 2.Defensive df found and saved')

        df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 3.Misc df found and saved')

        df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 4.Tracking df found and saved')

        df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 5.Usage df found and saved')
        
        df_traditional = BoxScoreTraditionalV2(game_id=game).get_data_frames()[0]
        time.sleep(1)
        print(f' 6.Traditional df found and saved')

#         x, = np.where(unique_array == game)
        x, = np.where(test_array == game)
#         x, = np.where(missing_games == game)

        game_ok_count += 1

        print(f'--- {x+1} games searched ---')
        print(f'--- {game_ok_count} games found ---')

        time.sleep(5)

        df_adv_list.append(df_advanced)
        df_def_list.append(df_defensive)
        df_misc_list.append(df_misc)
        df_tracking_list.append(df_tracking)
        df_usage_list.append(df_usage)
        df_traditional_list.append(df_traditional)

        print(f'--- {game_ok_count} games appended ---')

        print(f'--- Time: {datetime.now() - startTime} ---')

    except:
    
    #   problem list
        game_no_count += 1

        game_problem_list.append(game)

        print(f'game {game} encountered timeout')
        print(f'--- {game_no_count} games with problems ---')

    #   sleep
        print(f'--- sleeping for 300 seconds ---')
        time.sleep(300)

    #   try again to search
        try:

            print(f'Searching again for game: {game}...')

            df_advanced = BoxScoreAdvancedV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 1.Advanced df found and saved')
        
            df_defensive = BoxScoreDefensive(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 2.Defensive df found and saved')

            df_misc = BoxScoreMiscV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 3.Misc df found and saved')

            df_tracking = BoxScorePlayerTrackV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 4.Tracking df found and saved')

            df_usage = BoxScoreUsageV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 5.Usage df found and saved')
            
            df_traditional = BoxScoreTraditionalV2(game_id=game).get_data_frames()[0]
            time.sleep(1)
            print(f' 6.Traditional df found and saved')

            x, = np.where(unique_array == game)

            game_ok_count += 1

            print(f'--- {x+1} games searched ---')
            print(f'--- {game_ok_count} games found ---')

            time.sleep(5)

            df_adv_list.append(df_advanced)
            df_def_list.append(df_defensive)
            df_misc_list.append(df_misc)
            df_tracking_list.append(df_tracking)
            df_usage_list.append(df_usage)
            df_traditional_list.append(df_traditional)

            print(f'--- {game_ok_count} games appended ---')

            print(f'--- Time: {datetime.now() - startTime} ---')

        except:

            print(f'game {game} encountered another timeout')
            print(f'--- {game_no_count} games with problems ---')

        #   sleep
            print(f'--- sleeping for 300 seconds ---')
            time.sleep(300)

            continue

    continue

df_adv_all = pd.concat(df_adv_list)
df_def_all = pd.concat(df_def_list)
df_misc_all = pd.concat(df_misc_list)
df_tracking_all = pd.concat(df_tracking_list)
df_usage_all = pd.concat(df_usage_list)
df_traditional_all = pd.concat(df_traditional_list)

# df_def_missing = pd.concat(df_def_list)
# df_misc_missing = pd.concat(df_misc_list)
# df_tracking_missing = pd.concat(df_tracking_list)
# df_usage_missing = pd.concat(df_usage_list)

print(f'--- merged df ---')

Searching for game: 0021900002...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [1] games searched ---
--- 1 games found ---
--- 1 games appended ---
--- Time: 0:00:29.355504 ---
Searching for game: 0021900001...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [2] games searched ---
--- 2 games found ---
--- 2 games appended ---
--- Time: 0:00:59.573359 ---
Searching for game: 0021900008...
 1.Advanced df found and saved
 2.Defensive df found and saved
 3.Misc df found and saved
 4.Tracking df found and saved
 5.Usage df found and saved
 6.Traditional df found and saved
--- [3] games searched ---
--- 3 games found ---
--- 3 games appended ---
--- Time: 0:01:29.462896 ---
Searching for game: 0021900010...
 1.Adv

In [79]:
df_adv_all.to_csv('current_advanced.csv')
df_def_all.to_csv('current_defensive.csv')
df_misc_all.to_csv('current_misc.csv')
df_tracking_all.to_csv('current_tracking.csv')
df_usage_all.to_csv('current_usage.csv')
df_traditional_all.to_csv('current_traditional.csv')

In [80]:
stats_df = df_def_all.merge(df_adv_all, 
                            on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [81]:
stats_df2 = stats_df.merge(df_misc_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [82]:
stats_df3 = stats_df2.merge(df_tracking_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [83]:
stats_df4 = stats_df3.merge(df_usage_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [84]:
stats_df5 = stats_df4.merge(df_traditional_all, on=['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
                           'PLAYER_NAME', 'START_POSITION', 'COMMENT',],
                            how='inner')

In [92]:
list(stats_df5.keys())

['AST_PCT',
 'AST_RATIO',
 'AST_TOV',
 'AST_x',
 'AST_y',
 'BLK',
 'BLKA',
 'BLK_x',
 'BLK_y',
 'CFGA',
 'CFGM',
 'CFG_PCT',
 'COMMENT',
 'DEF_RATING',
 'DFGA',
 'DFGM',
 'DFG_PCT',
 'DIST',
 'DRBC',
 'DREB_PCT',
 'DREB_x',
 'DREB_y',
 'EFG_PCT',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'E_USG_PCT',
 'FG3A',
 'FG3M',
 'FG3_PCT',
 'FGA',
 'FGM',
 'FG_PCT_x',
 'FG_PCT_y',
 'FTA',
 'FTAST',
 'FTM',
 'FT_PCT',
 'GAME_ID',
 'MATCHUP_AST',
 'MATCHUP_FG3A',
 'MATCHUP_FG3M',
 'MATCHUP_FG3_PCT',
 'MATCHUP_FGA',
 'MATCHUP_FGM',
 'MATCHUP_FG_PCT',
 'MATCHUP_MIN',
 'MATCHUP_TOV',
 'MIN',
 'MIN_x',
 'MIN_x',
 'MIN_y',
 'MIN_y',
 'NET_RATING',
 'OFF_RATING',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_PAINT',
 'ORBC',
 'OREB',
 'OREB_PCT',
 'PACE',
 'PACE_PER40',
 'PARTIAL_POSS',
 'PASS',
 'PCT_AST',
 'PCT_BLK',
 'PCT_BLKA',
 'PCT_DREB',
 'PCT_FG3A',
 'PCT_FG3M',
 'PCT_FGA',
 'PCT_FGM',
 'PCT_FTA',
 'PCT_FTM',
 'PCT_OREB',
 'PCT_PF',
 'PCT_PFD',
 'PCT_PT

In [108]:
stats_df5.sort_index(axis='columns', inplace=True)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [110]:
stats_df5.to_csv('current_all_boxscores.csv')

#### cleanup, add gamescore, etc

In [114]:
df_boxscores = pd.read_csv('current_all_boxscores.csv')

In [115]:
df_boxscores

,AST_PCT,AST_RATIO,AST_TOV,AST,BLK,BLKA,CFGA,CFGM,CFG_PCT,COMMENT,...,TEAM_CITY,TEAM_ID,TEAM_NICKNAME,TM_TOV_PCT,TO,TS_PCT,UFGA,UFGM,UFG_PCT,USG_PCT
0,0.227,14.3,1.67,5,2,1,11,5,0.455,NaN,...,Los Angeles,1610612747,Lakers,8.6,3,0.460,10,3,0.300,0.345
1,0.444,23.5,1.60,8,1,2,8,4,0.500,NaN,...,Los Angeles,1610612747,Lakers,14.7,5,0.434,11,3,0.273,0.302
2,0.000,0.0,0.00,0,2,0,3,2,0.667,NaN,...,Los Angeles,1610612747,Lakers,25.0,1,0.667,0,0,0.000,0.103
3,0.000,0.0,0.00,0,1,0,4,3,0.750,NaN,...,Los Angeles,1610612747,Lakers,0.0,0,0.970,10,7,0.700,0.182
4,0.000,0.0,0.00,0,0,0,3,1,0.333,NaN,...,Los Angeles,1610612747,Lakers,22.2,2,0.571,4,2,0.500,0.170
5,0.125,42.9,3.00,3,0,0,0,0,0.000,NaN,...,Los Angeles,1610612747,Lakers,14.3,1,0.000,3,0,0.000,0.058
6,0.059,25.0,0.00,1,1,2,3,1,0.333,NaN,...,Los Angeles,1610612747,Lakers,0.0,0,0.436,0,0,0.000,0.068
7,0.125,12.5,1.00,1,0,0,1,1,1.000,NaN,...,Los Angeles,1610612747,Lakers,12.5,1,0.466,5,1,0.200,0.206
8,0.167,20.0,2.00,2,0,0,3,1,0.333,NaN,...,Los Angeles,1610612747,Lakers,10.0,1,0.286,-3,-1,0.333,0.182
9,0.000,0.0,0.00,0,0,0,0,0,0.000,NaN,...,Los Angeles,1610612747,Lakers,0.0,0,1.500,2,2,1.000,0.065


In [116]:
list(df_boxscores.keys())

['AST_PCT',
 'AST_RATIO',
 'AST_TOV',
 'AST',
 'BLK',
 'BLKA',
 'CFGA',
 'CFGM',
 'CFG_PCT',
 'COMMENT',
 'DEF_RATING',
 'DFGA',
 'DFGM',
 'DFG_PCT',
 'DIST',
 'DRBC',
 'DREB_PCT',
 'DREB',
 'EFG_PCT',
 'E_DEF_RATING',
 'E_NET_RATING',
 'E_OFF_RATING',
 'E_PACE',
 'E_USG_PCT',
 'FG3A',
 'FG3M',
 'FG3_PCT',
 'FGA',
 'FGM',
 'FG_PCT',
 'FTA',
 'FTAST',
 'FTM',
 'FT_PCT',
 'GAME_ID',
 'MATCHUP_AST',
 'MATCHUP_FG3A',
 'MATCHUP_FG3M',
 'MATCHUP_FG3_PCT',
 'MATCHUP_FGA',
 'MATCHUP_FGM',
 'MATCHUP_FG_PCT',
 'MATCHUP_MIN',
 'MATCHUP_TOV',
 'MIN',
 'NET_RATING',
 'OFF_RATING',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_PAINT',
 'ORBC',
 'OREB',
 'OREB_PCT',
 'PACE',
 'PACE_PER40',
 'PARTIAL_POSS',
 'PASS',
 'PCT_AST',
 'PCT_BLK',
 'PCT_BLKA',
 'PCT_DREB',
 'PCT_FG3A',
 'PCT_FG3M',
 'PCT_FGA',
 'PCT_FGM',
 'PCT_FTA',
 'PCT_FTM',
 'PCT_OREB',
 'PCT_PF',
 'PCT_PFD',
 'PCT_PTS',
 'PCT_REB',
 'PCT_STL',
 'PCT_TOV',
 'PFD',
 'PF',
 'PIE',
 'PLAYER_ID',
 'PLAYER_NAME',
 'PLAYE

In [125]:
df_boxscores['GAMESCORE'] = 0
for index,row in df_boxscores.iterrows():
    
    pts = df_boxscores.loc[index, 'PTS']
    fgm = df_boxscores.loc[index, 'FGM']
    oreb = df_boxscores.loc[index, 'OREB']
    dreb = df_boxscores.loc[index, 'DREB']
    stl = df_boxscores.loc[index, 'STL']
    ast = df_boxscores.loc[index, 'AST']
    blk = df_boxscores.loc[index, 'BLK']
    fga = df_boxscores.loc[index, 'FGA']
    fta = df_boxscores.loc[index, 'FTA']
    ftm = df_boxscores.loc[index, 'FTM']
    ftmiss = fta - ftm
    pf = df_boxscores.loc[index, 'PF']
    tov = df_boxscores.loc[index, 'TO']
    
    gamescore = round((
            (pts) +
            (0.4*fgm) +
            (0.7*oreb) +
            (0.3*dreb) +
            (stl) +
            (0.7*ast) +
            (0.7*blk) -
            (0.7*fga) -
            (0.4*ftmiss) -
            (0.4*pf) -
            (tov)
    ), 3)
    
    print(gamescore)
    
    df_boxscores.loc[index, 'GAMESCORE'] = gamescore
    

17.1
11.6
4.1
26.2
2.0
-2.1
4.1
1.9
0.6
4.6
20.7
2.4
6.7
5.5
2.6
13.3
20.2
8.0
12.1
5.9
15.7
4.6
9.2
7.1
13.7
6.6
3.6
-0.4
4.7
12.4
4.5
6.7
25.4
0.8
14.8
32.2
6.7
2.3
6.5
17.2
10.0
9.6
4.3
-1.2
4.5
2.9
-0.4
0.6
-1.2
-0.7
4.7
13.8
9.6
13.3
13.0
19.9
-2.3
1.3
2.8
4.0
4.7
0.7
0.0
23.9
18.9
8.4
-3.1
15.4
-0.9
-1.9
7.2
-0.1
15.5
-2.1
5.1
16.4
7.5
15.8
19.0
5.5
10.9
7.0
1.6
3.1
3.8
9.3
1.7
16.6
0.3
-4.4
-0.4
8.0
3.8
5.6
2.3
0.9
0.3
16.5
5.4
19.3
17.3
13.3
9.1
1.4
6.0
7.8
8.3
1.2
-0.7
2.7
7.1
9.6
31.4
7.5
10.5
2.8
3.5
1.3
11.9
8.3
9.8
15.7
8.9
10.2
40.5
0.4
0.7
0.8
3.0
-1.9
10.6
12.0
15.2
5.4
6.1
4.0
3.8
1.4
-2.3
3.9
6.2
18.1
10.5
8.7
8.8
3.2
8.5
4.3
1.2
9.4
8.5
0.6
3.2
5.3
8.9
11.8
3.6
0.1
4.8
0.6
1.6
20.8
11.3
-1.1
9.9
17.1
14.9
0.4
6.9
-0.9
8.2
-0.7
0.0
1.9
17.4
-0.9
13.2
17.6
-0.0
3.0
-2.3
5.7
3.4
9.7
12.1
3.3
27.9
-5.7
2.7
10.7
2.0
1.3
0.3
18.8
10.7
14.3
-2.6
10.1
8.7
1.8
7.3
10.1
4.0
7.0
3.4
17.0
0.0
24.9
-2.1
4.7
7.3
0.3
0.7


In [127]:
df_boxscores.to_csv('current_all_boxscores.csv')